In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab09.ipynb")

# Lab 9: AES Modes of Operation
Contributions From: Ryan Cottone

Welcome to Lab 9! In this lab, we will be exploring the basics of block and stream ciphers, as well as different modes of operation for AES.

In [ ]:
import numpy as np

In [ ]:
%%capture
import sys
!{sys.executable} -m pip install pycryptodome

In [ ]:
from Crypto.Cipher import AES
from Crypto.Hash import SHA256

# Block Ciphers

Up to this point we have primarily been concerned with elementary ciphers or asymmetric encryption. Real-world symmetric encryption is most often accommplished by **block ciphers**, which operate on fixed-length blocks of data, much like our Merkle-Damgard hash functions did. We can model this as some function $f(k, d) = c$, taking in the key and some data $d$ to output some ciphertext $c$.

### Advanced Encryption Standard

The most popular symmetric cryptosystem in the world, AES (aka Rijndael), was first published in 1998. Since then, it has become the premier block cipher, used even by the NSA for top-secret data. To date, there are no real-world attacks against correctly-implemented AES.

We will explore how to use AES in this lab!

### Modes of Operation

AES itself only specifies $f(k, d)$ for us, meaning we have to figure out how to encrypt the data blocks $P_1, P_2, \ldots P_k$ into ciphertext. The different ways of doing this are called **modes of operation**.


## AES-ECB (Electronic Code Book)

The most straightforward way to encrypt using AES is to set $C_i = \text{Enc(}k, P_i\text{)} $, in effect each ciphertext block being the direct encryption of its corresponding plaintext. This can be visualized by the following (credit to the [CS 161 Textbook](https://textbook.cs161.org/crypto/symmetric.html)):

![AES-ECB Encryption](https://textbook.cs161.org/assets/images/crypto/symmetric/ECB_encryption.png)
![AES-ECB Decryption](https://textbook.cs161.org/assets/images/crypto/symmetric/ECB_decryption.png)

**Question 1**: Implement the ECB mode of operation!

In [ ]:
# This code is given to you

def blockEncrypt(key, data): # Takes in 16-byte key and data and returns their encryption.
    if len(key) != 16 or len(data) != 16:
        raise AssertionError("Key and plaintext must be 16 bytes long")
    
    cipher = AES.new(key, AES.MODE_ECB) # Ensure consistent behavior on one-block encryptions
    
    return cipher.encrypt(data)

def blockDecrypt(key, data): # Takes in 16-byte key and data and returns their encryption.
    if len(key) != 16 or len(data) != 16:
        raise AssertionError("Key and plaintext must be 16 bytes long")
    
    cipher = AES.new(key, AES.MODE_ECB) # Ensure consistent behavior on one-block encryptions
    
    return cipher.decrypt(data)

In [ ]:
def AES_ECB_encrypt(key, dataBlocks): # Key is a 16 byte bytestring, dataBlocks is an array of 16 byte bytestrings.
    if len(dataBlocks) == 0:
        return []
    if len(key) != 16:
        raise AssertionError("Incorrect key length")
    
    cipherBlocks = []
    
    for block in dataBlocks:
        if len(block) != 16:
            raise AssertionError("Incorrect block length")
        
        # Remember you have blockEncrypt(key, dataBlock)!
        ...
    
    return cipherBlocks

def AES_ECB_decrypt(key, cipherBlocks): # Key is a 16 byte bytestring, dataBlocks is an array of 16 byte bytestrings.
    if len(cipherBlocks) == 0:
        return []
    if len(key) != 16:
        raise AssertionError("Incorrect key length")
    
    dataBlocks = []
    
    for block in cipherBlocks:
        if len(block) != 16:
            raise AssertionError("Incorrect block length")
        
        # Remember you have blockEncrypt(key, dataBlock)!
        ...
    
    return dataBlocks

In [ ]:
grader.check("q1_1")

### AES-ECB Insecurity

AES-ECB is actually terribly insecure, not because it leaks your plaintext, but rather because it leaks indirect information about it. Since we encrypt without a source of randomness, the encryption is wholly **deterministic**. Moreover, since we encrypt each block independently, it is possible to see which underlying plaintext blocks are the same (or different) when compared two ciphertexts. One must simply check if $C_i = C_i'$.

**Question 1.2**: Implement a function that outputs the blocks of plaintext which are same for two separate AES-ECB ciphertexts.

In [ ]:
def detectIdenticalBlocks(c_one, c_two):
    identical = []
    
    for i in range(min(len(c_one), len(c_two))):
        ...
            identical.append(i)
    
    return identical

In [ ]:
grader.check("q1_2")

## AES-CBC (Cipher Block Chaining)

A much more secure (and widely used) mode of operation for AES is **AES-CBC**. This mode fixes the weaknesses of ECB by introduces a source of deterministic randomness at the start, and "carrying forward" that randomness throughout the cipher. Specifically, the encryption of block i is now $$C_i = \text{Enc(}k, P_i \oplus C_{i-1}\text{)}$$ and decryption is correspondingly $$P_i = \text{Dec(}k, C_i\text{)} \oplus C_{i-1}$$

But what is $C_0$? $C_0$ is defined as the **initialization vector**, which is randomly generated for every new encryption. It is published alonside the ciphertext as the first block. (Exercise: what would happen if we used the same IV for two similar messages?)

A visual aid courtesy of CS 161:
    
![AES-CBC Encryption](https://textbook.cs161.org/assets/images/crypto/symmetric/CBC_encryption.png)
![AES-CBC Decryption](https://textbook.cs161.org/assets/images/crypto/symmetric/CBC_decryption.png)

**Question 2.1**: Implement AES-CBC!

In [ ]:
def bxor(b1, b2): # use xor for bytes
    result = bytearray()
    for b1, b2 in zip(b1, b2):
        result.append(b1 ^ b2)
    return result

In [ ]:
def AES_CBC_encrypt(key, IV, dataBlocks): # Key/IV are 16 byte bytestrings, dataBlocks is an array of 16 byte bytestrings.
    if len(dataBlocks) == 0:
        return []
    if len(key) != 16:
        raise AssertionError("Incorrect key length")
    
    cipherBlocks = [IV]
    
    lastCipherBlock = IV
    
    for block in dataBlocks:
        if len(block) != 16:
            raise AssertionError("Incorrect block length")
        
        # Remember you have bxor(b1, b2) and blockEncrypt(key, dataBlock)!
        encInput = ...
        ...
        lastCipherBlock = cipherBlocks[-1]
    
    return cipherBlocks

def AES_CBC_decrypt(key, cipherBlocks): # Key is a 16 byte bytestring, dataBlocks is an array of 16 byte bytestrings.
    if len(cipherBlocks) == 0:
        return []
    if len(key) != 16:
        raise AssertionError("Incorrect key length")
        
    lastCipherBlock = cipherBlocks[0] # IV is the zero-th ciphertext block!
    
    dataBlocks = []
    
    for block in cipherBlocks[1:]:
        if len(block) != 16:
            raise AssertionError("Incorrect block length")
        
        # Remember you have blockEncrypt(key, dataBlock)!
        decrypted = ...
        xored = ...
        dataBlocks.append(xored)
        lastCipherBlock = block
    
    return dataBlocks

In [ ]:
grader.check("q2_1")

## Stream Ciphers

Unlike block ciphers, stream ciphers operate on arbitrary-length bitstrings that aren't all known at once. A common example of this would be encrypted phone calls, since we have to encrypt them on the fly. This is possible, but messy, to do with block ciphers. Stream ciphers would ideally output some function $f(b)$ that takes in bits $b$ and outputs a ciphertext of the same length. First, we must initialize it with a key, however.

While there are sophisticated stream ciphers outside of the scope of this class, we can use a simple example for education purposes. Remember the **one-time pad**? Where we XORed the plaintext bits with some key bits? That would be the perfect stream cipher, if not for the difficulty of producing so much key material. Instead, we can generate the key material on the fly using a CSPRNG!

(Note: actual stream ciphers are much more complicated and have to deal with stuff like synchronization. Please do not attempt to build your own encryption systems for anything even remotely important!)

In our example system, we find $f(b) = b \oplus \text{CSPRNG(b)}$, where CSPRNG is a seeded CSPRNG that takes in $b$ and returns $b$ pseudorandom bits. Note that decryption of a stream cipher output is just running the stream cipher again with a fresh CSPRNG!

In [ ]:
# These functions are all given to you
def H(x):
    if type(x) != bytes:
        if type(x) == str:
            x = bytes(x, 'utf-8')
        elif type(x) == int:
            x = x.to_bytes(len(bin(x))-2, byteorder='big')
    
    h = SHA256.new()
    h.update(x)
    return h.digest().hex()

def hashPRNG(seed):
    state = seed
    
    def export(bits):
        nonlocal state
        assert bits <= 256, "Can only output at most 256 bits"
        state = int(H(state),16)  
        return state & ((1 << bits) - 1)
    
    return export

def getExpansion(n,m):
    arr = []
    
    while n > 0:
        r = n % m
        n //= m
        
        arr.append(r)
    
    return arr


def textToInt(s):
    total = 0
    
    for i in range(len(s)):
        total += ord(s[i])*(256**i)
    
    return total

def intToText(n):
    expansion = getExpansion(n, 256)
    
    finalStr = ""
    
    for i in range(len(expansion)):
        finalStr += chr(expansion[i])
        
    return finalStr

**Question 3.1**: Implement our example stream cipher!

In [ ]:
def streamCipher(key): # Takes in bytestring key
    csprng = hashPRNG(key)
    
    # csprng(b) outputs b psuedorandom bits
    
    def export(data):
        # Takes in integer data to encrypt and returns the ciphertext
        nonlocal csprng

        keybits = csprng(len(bin(data)) - 2)
        
        ...

    return export

In [ ]:
grader.check("q3_1")

Congrats on finishing Lab 9!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have generated the zip file, go to the Gradescope page for this assignment to submit.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)